In [2]:
import pandas as pd
import altair as alt
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# The data is given in the form of an excel sheet 
# Open the excel sheet 
file_path = 'data/OECD_betterLifeIndex.xlsx'
sheet_name = 'PyhtonReadable'
df = pd.read_excel(file_path, sheet_name=sheet_name)
# Strip whitespace
df['Continent'] = df['Continent'].str.replace(r'\s', '').str.strip()
df
df.columns = df.columns.str.strip()

In [3]:
df.replace('..', np.nan, inplace=True)
ignore = ['Country', 'OCED', 'Continent']
# Adjust this so that countries are split by continent :~)
# todo: change this to adjust for continent
for column in df.columns:
    if column not in ignore:
        column_mean = df[column].mean()
        column_mean = round(column_mean, 2)
        print(f'{column} : {column_mean}')
        df[column].fillna(column_mean, inplace=True)

df

Dwellings without basic facilities : 4.26
Housing expenditure : 20.35
Rooms per person : 1.66
Household net adjusted disposable income : 29598.58
Household net wealth : 323959.63
Labour market insecurity : 5.18
Employment Rate : 67.64
Long-term unemployment rate : 2.11
Personal earnings : 43933.06
Quality of support network : 90.79
Educational attainment : 78.12
Student skills : 484.2
Years in education : 17.5
Air pollution : 13.52
Water quality : 83.33
Stakeholder engagement for developing regulations : 2.15
Voter turnout : 69.26
Life expectancy : 80.28
Self-reported health : 67.95
Life satisfaction : 6.6
Feeling safe walking alone at night : 72.12
Homicide rate : 3.27
Employees working very long hours : 7.26
Time devoted to leisure and personal care : 15.07


,Continent,Country,Dwellings without basic facilities,Housing expenditure,Rooms per person,Household net adjusted disposable income,Household net wealth,Labour market insecurity,Employment Rate,Long-term unemployment rate,...,Stakeholder engagement for developing regulations,Voter turnout,Life expectancy,Self-reported health,Life satisfaction,Feeling safe walking alone at night,Homicide rate,Employees working very long hours,Time devoted to leisure and personal care,OCED
0,Oceania,Australia,4.26,19.40,1.66,37433.00,528768.00,3.10,73,1.00,...,2.70,92,83.0,85.00,7.1,67,0.9,12.50,14.36,1.0
1,Europe,Austria,0.80,20.80,1.60,37001.00,309637.00,2.30,72,1.30,...,1.30,76,82.0,71.00,7.2,86,0.5,5.30,14.51,1.0
2,Europe,Belgium,0.70,20.00,2.10,34884.00,447607.00,2.40,65,2.30,...,2.00,88,82.1,74.00,6.8,56,1.1,4.30,15.52,1.0
3,North America,Canada,0.20,22.90,2.60,34421.00,478240.00,3.80,70,0.50,...,2.90,68,82.1,89.00,7.0,78,1.2,3.30,14.57,1.0
4,South America,Chile,9.40,18.40,1.90,29598.58,135787.00,7.00,56,2.11,...,1.30,47,80.6,60.00,6.2,41,2.4,7.70,15.07,1.0
5,South America,Colombia,12.30,20.35,1.00,29598.58,323959.63,5.18,58,1.10,...,1.40,53,76.7,80.00,5.7,50,23.1,23.70,15.07,1.0
6,North America,Costa Rica,2.30,17.00,1.20,16517.00,323959.63,5.18,55,1.50,...,1.80,66,80.5,73.00,6.3,47,10.0,22.00,15.07,1.0
7,Europe,Czech Republic,0.50,23.40,1.50,26664.00,323959.63,2.30,74,0.60,...,1.60,62,79.3,62.00,6.9,77,0.7,4.50,15.07,1.0
8,Europe,Denmark,0.50,23.30,1.90,33774.00,149864.00,4.50,74,0.90,...,2.00,85,81.5,70.00,7.5,85,0.5,1.10,15.07,1.0
9,Europe,Estonia,5.70,17.00,1.70,23784.00,188627.00,5.40,74,1.20,...,2.70,64,78.8,57.00,6.5,79,1.9,2.20,14.98,1.0


In [4]:
# load geo json for map display view
world_geo = gpd.read_file('data/custom.geo.json')
# merge data from excel with geo_json data
merged_data = world_geo.merge(df, how='left', left_on='name_en', right_on='Country')

In [ ]:
gdf_sel = merged_data.query("Continent == 'Asia'")

# Define brush
brush = alt.selection_point()



hist = alt.Chart(merged_data).mark_bar().encode(
    y='Homicide rate',
    x='Country:N',
    color='Country:N'
).transform_filter(
    brush
).add_params(
    brush
).properties(
    width=600,
    height=300
)



# Slider variables
range0 = alt.binding_range(min=-180, max=180, step=5, name='rotate longitude ')
range1 = alt.binding_range(min=-180, max=180, step=5, name='rotate latitude ')
range2 = alt.binding_range(min=100, max=300, step=5, name='Zoom')
rotate0 = alt.param(value=-10, bind=range0)
rotate1 = alt.param(value=-20, bind=range1)
rotate2 = alt.param(value=100, bind=range2)
hover = alt.selection_point(on="mouseover", clear="mouseout")
column = alt.selection_point(
    fields=df.columns
    )
# Background sphere
sphere = alt.Chart(alt.sphere()).mark_geoshape(
    fill="aliceblue", stroke="black", strokeWidth=1.5
)
# This is the actual globe that will display the country data
world = alt.Chart(merged_data).mark_geoshape(
     stroke="black", strokeWidth=0.35
).encode(
    # Create an interactive tooltip
    tooltip=['Country:N', 'Homicide rate:N', 'Continent:N'],
    stroke=alt.condition(brush, alt.value('red'), alt.value('gray')),
    color=alt.condition(
        alt.datum['Homicide rate'] < 0,
        alt.value('gray'),
        alt.Color('Homicide rate')
    )
).properties(
    width=300,
    height=300
).add_params(
    brush
).add_params(hover, rotate0,rotate1,rotate2, column)
# Layer together the background and world data
map = alt.layer(sphere, world).project(
    type="orthographic",
    scale=rotate2, # When this is set, it removes  the default scale params which is what is moving the sphere off centre
    rotate=alt.expr(f"[{rotate0.name}, {rotate1.name}, 0]"), # Scale the spheres rotation based on the sliders defined above
    # This moves our sphere down so that it is not off screen
    translate=[150,150]
)
test = alt.hconcat(map, hist, title='Aren\'t I great' )
test.save('test.html')